# Reddit: Biden is Dead

This notebook is a little collaboration with [Joan](https://twitter.com/bostonjoan) to collect posts from Reddit in the last few days that mention that Biden is dead or has COVID-19. We're going to use the PushShift API since it can return more than the last 100 results.

The PushShift API has [quite a few knobs](https://github.com/pushshift/api#search-parameters-for-submissions) for searching submissions. We're going to use `before` and `after` to walk back in time, and `title` to just search the title of the posts. 

In [47]:
url = 'https://api.pushshift.io/reddit/search/submission'
params = {
    'title': 'biden dead',
    'limit': 1000
}

To give you a sense of the type of data that PushShift make available for each post lets just display one.

In [49]:
import requests

requests.get(url, params=params).json()['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'jkv2',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_5xrgn3dl',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1584973911,
 'domain': 'self.AskReddit',
 'full_link': 'https://www.reddit.com/r/AskReddit/comments/fnkwd1/is_joe_biden_dead/',
 'gildings': {},
 'id': 'fnkwd1',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'all_ads',
 'per

Since it would be useful to be able to look at *biden dead* as well as *biden coronavirus* lets create a function that will run a query back to a given datetime.

In [142]:
import time

def pushshift(q, begin):
    stop = False
    before = 0
    after = 1
    while stop is not True:
        params['title'] = q
        params['before'] = '{}d'.format(before)
        params['after'] = '{}d'.format(after)
        resp = requests.get(url, params=params)
        results = resp.json()['data']
        if len(results) > 0:
            for post in results:
                created = datetime.datetime.fromtimestamp(post['created_utc'])
                post['created'] = created
                yield post
                if created <= begin:
                    stop = True
            before += 1
        after += 1
        time.sleep(1)

Let's test it out:

In [158]:
for post in pushshift('biden dead', datetime.datetime(2020, 3, 10)):
    print(post['title'])

If Joe Biden is dead that means that Bernie graduates with straight A’s
mr biden, your dead sir
Joe Biden Found Dead In Miami
Donald Trump has been on TV for like four hours yelling at the press and Joe Biden can’t even stand for seven minutes cause he’s dead.
Proof Biden is dead.
Is Joe Biden dead?
I’ve seen posts about Joe Biden being dead, can someone explain? I’m a bit out of the loop lol
Biden isn't dead you idiots, it's obvious what's happening:
If Biden isn't dead, it further proves that he's apathetic. He doesn't care about our collective misery and suffering and, if he isn't dead and somehow wins this General Election in 2020, his Presidency is going to be fucking awful.
is joe biden dead?
STOP WITH THE JOE BIDEN IS DEAD MEMES
If Joe Biden is dead, Hunter should run in his place
MMW: At time of Writing, Joseph R. Biden Jr has been dead for more than 24 hours.
Joe Biden is dead
Joe Biden isn't dead you morons
Joe Biden isn't dead you morons
Joe Biden IS dead.
I’ll vote Biden if

Biden and Bernie to Pennsylvania, Ohio and West Virginia: Drop Dead | RealClearPolitics
Isn’t it funny that all these old fucks voting for Biden will be dead from the corona virus by the General election.
Don’t even think about it Biden, you’ll be dead by two weeks after your inauguration. #BidenVP2020
How in the gel did Joe Biden go from about dead last to ahead by a mile?
Bro's it's over.... Joe Biden has raped us 2 Super Tuesdays in a row.... The revolution is dead... The Establishment has Won... Our Beta Male energy wasn't enough... Black people weren't smart enough to vote for the revolution... All we can do now is vote for Trump..... I'm going to cry.....😭😭😭.
Bro's it's over.... Joe Biden has raped us 2 Super Tuesdays in a row.... The revolution is dead... The Establishment has Won... Our Beta Male energy wasn't enough... Black people weren't smart enough to vote for the revolution... All we can do now is vote for Trump..... I'm going to cry.....😭😭😭.
Veteran confronts Joe Biden f

## Biden is Dead?

Now let's use our pushshift function to write out the results to a CSV.

In [159]:
import csv

with open('data/biden-dead.csv', 'w') as fh:
    out = csv.DictWriter(fh, extrasaction='ignore', fieldnames=['id', 'url', 'created', 'title', 'score', 'num_comments', 'subreddit'])
    out.writeheader()
    for post in pushshift('biden dead', datetime.datetime(2020, 3, 10)):
        out.writerow(post)

We can load them into pandas to sort 'em. But first a little trick to prevent word-wrapping in dataframe output:

In [160]:
pandas.set_option('display.max_columns', None)
pandas.set_option('display.width', None)
pandas.set_option('display.max_colwidth', -1)

In [161]:
import pandas 

dead = pandas.read_csv('data/biden-dead.csv', parse_dates=['created'])
dead = dead.sort_values('created', ascending=True)
dead.head(10)

,id,url,created,title,score,num_comments,subreddit
145,fg3qvt,https://youtu.be/Q8f-XUyfXCg,2020-03-09 19:03:41,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,3,dsa
146,fg3qyg,https://youtu.be/Q8f-XUyfXCg,2020-03-09 19:03:50,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,0,PoliticalVideos
147,fg3r1l,https://youtu.be/Q8f-XUyfXCg,2020-03-09 19:03:58,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,0,TYTnetwork
148,fg3v33,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 19:11:17,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote - Secular Talk,1,0,PoliticalVideos
149,fg42u8,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 19:25:22,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,0,jimmydoreshow
150,fg5jul,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 21:05:04,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,9,ChapoTrapHouse
151,fg5jyd,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 21:05:16,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,1,WayOfTheBern
152,fg6943,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 21:54:49,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,96,SandersForPresident
153,fg7nae,https://www.youtube.com/watch?v=Q8f-XUyfXCg,2020-03-09 23:37:17,US Veteran Tells Biden 'My Friends Are Dead' Bc Of Your Iraq War Vote,1,39,Political_Revolution
154,fgb9e3,https://i.redd.it/ts3kcz2wftl41.jpg,2020-03-10 05:40:46,Bernie Bros.... I think we're finished.... The Establishment is too powerful... Biden has too much black energy behind him... We're to white bro's.... We're 2 rational.... The revolution is dead...,1,0,BernieSanders


Ok, so what was the first post that contained the phrase *biden is dead*?

In [162]:
import re

exact_dead = dead[dead.title.str.match(re.compile('.*biden is dead.*', re.IGNORECASE))]
exact_dead = exact_dead.sort_values('created', ascending=True)
exact_dead.head(10)

,id,url,created,title,score,num_comments,subreddit
68,fku1b3,https://www.reddit.com/r/ChapoTrapHouse/comments/fku1b3/joe_biden_is_dead/,2020-03-18 14:05:09,Joe Biden is dead,7,8,ChapoTrapHouse
63,fku1b3,https://www.reddit.com/r/ChapoTrapHouse/comments/fku1b3/joe_biden_is_dead/,2020-03-18 14:05:09,Joe Biden is dead,7,8,ChapoTrapHouse
59,fmkydt,https://www.reddit.com/r/TooAfraidToAsk/comments/fmkydt/why_are_people_sayong_that_biden_is_dead/,2020-03-21 15:46:51,Why are people sayong that Biden is dead?,1,5,TooAfraidToAsk
27,fmpgvt,https://www.reddit.com/r/redscarepod/comments/fmpgvt/joe_biden_is_dead/,2020-03-21 19:31:10,Joe Biden is Dead,1,0,redscarepod
36,fmwz65,https://i.redd.it/se1z23rtv6o41.jpg,2020-03-22 05:01:31,They are trying to hide the fact that Joe Biden is dead. Typical simp shit,1,1,ChapoTrapHouse
42,fn31bq,https://www.reddit.com/r/unpopularopinion/comments/fn31bq/joe_biden_is_dead/,2020-03-22 13:09:44,Joe Biden is dead.,1,2,unpopularopinion
43,fn3z9s,https://twitter.com/NathanJRobinson/status/1241567076126523393,2020-03-22 14:04:05,"@NathanJRobinson I don't actually think Joe Biden is dead but I do think that at the very moment every person in the country is shitting themselves with fear, having the lead candidate absent for an entire week is a colossal abdication of his most basic job.",1,47,WayOfTheBern
44,fn458y,https://twitter.com/peterawolf/status/1241787238096285697,2020-03-22 14:12:52,Left Twitter: lol nobody will vote for Biden lol Biden: *wins SC* Left Twitter: lol Biden has dementia lol Biden: *wins debate* Left Twitter: lol Biden doesn't care about corona lol Biden: *arranges shadow health briefing* Left Twitter: lol Biden is dead lol Biden: *lives*,1,8,Enough_Sanders_Spam
45,fn48w1,https://twitter.com/NathanJRobinson/status/1241567076126523393,2020-03-22 14:18:07,"I don't actually think Joe Biden is dead but I do think that at the very moment every person in the country is shitting themselves with fear, having the lead candidate absent for an entire week is a colossal abdication of his most basic job.",1,2,FakeProgressives
46,fn4amo,https://twitter.com/peterawolf/status/1241787238096285697,2020-03-22 14:20:41,Left Twitter: lol nobody will vote for Biden lol Biden: *wins SC* Left Twitter: lol Biden has dementia lol Biden: *wins debate* Left Twitter: lol Biden doesn't care about corona lol Biden: *arranges shadow health briefing* Left Twitter: lol Biden is dead lol Biden: *lives*,1,0,JoeBiden


So it looks like the first mention of Joe Biden being dead (since 2018-03-10) was https://www.reddit.com/r/ChapoTrapHouse/comments/fku1b3/joe_biden_is_dead/ on 2020-03-18.

Of course you can do this in a spreadsheet program with the CSV, but we can also see what the most commented on thread was:

In [155]:
exact_dead = exact_dead.sort_values('num_comments', ascending=False)
exact_dead.head()

,id,url,created,title,score,num_comments,subreddit
44,fn3z9s,https://twitter.com/NathanJRobinson/status/1241567076126523393,2020-03-22 14:04:05,"@NathanJRobinson I don't actually think Joe Biden is dead but I do think that at the very moment every person in the country is shitting themselves with fear, having the lead candidate absent for an entire week is a colossal abdication of his most basic job.",1,47,WayOfTheBern
12,fndwyf,https://www.reddit.com/r/ChapoTrapHouse/comments/fndwyf/if_joe_biden_is_dead_hunter_should_run_in_his/,2020-03-23 00:28:34,"If Joe Biden is dead, Hunter should run in his place",1,37,ChapoTrapHouse
14,fnfo3s,https://www.reddit.com/r/ChapoTrapHouse/comments/fnfo3s/joe_biden_is_dead/,2020-03-23 03:04:00,Joe Biden is dead,1,10,ChapoTrapHouse
17,fnfx97,https://www.reddit.com/r/ChapoTrapHouse/comments/fnfx97/joe_biden_is_dead/,2020-03-23 03:28:56,Joe Biden IS dead.,1,9,ChapoTrapHouse
68,fku1b3,https://www.reddit.com/r/ChapoTrapHouse/comments/fku1b3/joe_biden_is_dead/,2020-03-18 14:05:09,Joe Biden is dead,7,8,ChapoTrapHouse


## Biden has Coronavirus

Similarly we can create a CSV for *biden coronavirus*.

In [156]:
import csv

with open('data/biden-coronavirus.csv', 'w') as fh:
    out = csv.DictWriter(fh, extrasaction='ignore', fieldnames=['id', 'url', 'created', 'title', 'score', 'num_comments', 'subreddit'])
    out.writeheader()
    for post in pushshift('biden coronavirus', datetime.datetime(2020, 3, 10)):
        out.writerow(post)

In [157]:
covid19 = pandas.read_csv('data/biden-coronavirus.csv', parse_dates=['created'])
covid19 = covid19.sort_values('created', ascending=True)
covid19

,id,url,created,title,score,num_comments,subreddit
2279,fg2581,https://twitter.com/realDonaldTrump/status/1237027356314869761?ref_src=twsrc%5Etfw%7Ctwcamp%5Etweetembed%7Ctwterm%5E1237027356314869761&amp;ref_url=https%3A%2F%2Fwww.redditmedia.com%2Fmediaembed%2Ffg121e%3Fresponsive%3Dtrue%26is_nightmode%3Dfalse,2020-03-09 17:22:57,"SO LISTEN UP FATASSES IF THE PRESIDENT IF A SENIOR CITIZEN HAS A 20% CHANCE AT BESS TO DIE FROM CORONAVIRUS, YOU THROW JOE BIDEN IN THE MIX THAT GIVES ME A 100% CHANCE TO WIN AT SACRIFICE HOLLA IF YA HEAR ME",1,15,ChapoTrapHouse
2280,fg2jtp,https://www.thenation.com/article/politics/sander-biden-trump-china-coronavirus/,2020-03-09 17:48:12,"“It’s time Democrats start paying attention to the gulf between Sanders and Biden (and Trump) on China. Sanders alone has always made building American manufacturing capacity central to his platform: a winning message, and one whose urgency is made all the more clear given the coronavirus.”",1,2,SandersForPresident
2281,fg2r5d,https://thehill.com/homenews/campaign/486615-biden-on-trumps-coronavirus-response-i-wish-he-would-just-be-quiet,2020-03-09 18:00:43,Biden on Trump's coronavirus response: 'I wish he would just be quiet' | TheHill - The Hill,1,0,nofeenews
2282,fg31on,https://www.reddit.com/r/AskReddit/comments/fg31on/serious_what_happens_with_china_if_coronavirus/,2020-03-09 18:18:52,"[Serious] What happens with China if Coronavirus kills President Trump, Bernie Sanders, or Joe Biden?",1,8,AskReddit
2283,fg35ew,https://www.reddit.com/r/askweb/comments/fg35ew/serious_what_happens_with_china_if_coronavirus/,2020-03-09 18:25:26,"[Serious] What happens with China if Coronavirus kills President Trump, Bernie Sanders, or Joe Biden?",1,0,askweb
...,...,...,...,...,...,...,...
139,fnrjme,https://www.nydailynews.com/coronavirus/ny-coronavirus-joe-biden-trump-20200323-xmttutgdmrcv5fmy4tuzxcynly-story.html,2020-03-23 16:43:14,Biden slams Trump for mishandling coronavirus response - New York Daily News,1,0,FreshNewsToday
140,fnrkdp,https://thehill.com/homenews/campaign/488966-biden-coronavirus-shows-why-conservatives-should-drop-obamacare-challenge,2020-03-23 16:44:23,Biden: Coronavirus shows why conservatives should drop ObamaCare challenge,1,0,u_LateRoyal
141,fnrlix,https://redstateofminddaily.com/biden-will-hold-shadow-briefings-on-coronavirus-to-contrast-from-president-trump-gets-blasted-in-response/,2020-03-23 16:46:16,"Biden Will Hold ‘Shadow Briefings’ on Coronavirus to Contrast From President Trump, Gets Blasted in Response",1,0,u_stratosphere53
142,fnrrcn,https://www.lifezette.com/2020/03/joe-biden-torches-trumps-coronavirus-response-if-youre-a-wartime-president-act-like-one/,2020-03-23 16:55:16,"Joe Biden torches Trump’s coronavirus response: If you’re a wartime president, ‘act like one’",1,0,TheNewsFeed
